In [1]:
from sleep_analysis.datasets.d04_main_dataset import D04MainStudy
import numpy as np
import pandas as pd
import os

In [2]:
from empkins_io.sync import SyncedDataset

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [3]:
from biopsykit.signals.ecg import EcgProcessor

In [4]:
from empkins_micro.emrad.radar import get_rpeaks, get_peak_probabilities

In [5]:
from fau_colors import cmaps, register_fausans_font

In [6]:
%matplotlib widget

In [7]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [8]:
### General Settings


# General Radar Settings
fs_radar = 1953.125
window_size = 120

# General PSG settings
clean_method = "biosppy"
peak_method = "neurokit"

In [9]:
def cut_signals(hr_ecg, hr_radar):
    start = max(hr_ecg.index[0], hr_radar.index[0])
    end = min(hr_ecg.index[-1], hr_radar.index[-1])
    display(start)
    display(end)
    return hr_ecg[start:end], hr_radar[start:end]

In [10]:
dataset = D04MainStudy()

In [1]:
dataset

NameError: name 'dataset' is not defined

In [12]:
def process_radar(synced_radar):
    print("-------------------------------------------------")
    print("Processing participant " + subj.index["subj_id"][0])
    print("-------------------------------------------------")

    
    print("Radar 1")
    lstm_output_1 = get_peak_probabilities(synced_radar["radar_1_resampled_"][["I", "Q"]], fs_radar=fs_radar, window_size=window_size)
    print("Radar 2")
    lstm_output_2 = get_peak_probabilities(synced_radar["radar_2_resampled_"][["I", "Q"]], fs_radar=fs_radar, window_size=window_size)
    print("Radar 3")
    lstm_output_3 = get_peak_probabilities(synced_radar["radar_3_resampled_"][["I", "Q"]], fs_radar=fs_radar, window_size=window_size)
    print("Radar 4")
    lstm_output_4 = get_peak_probabilities(synced_radar["radar_4_resampled_"][["I", "Q"]], fs_radar=fs_radar, window_size=window_size)

    return {"lstm_output_1" : lstm_output_1, "lstm_output_2" : lstm_output_2, "lstm_output_3" : lstm_output_3, "lstm_output_4" : lstm_output_4}

    

In [13]:
from biopsykit.utils.exceptions import EcgProcessingError

In [14]:
def get_MAE_results(probability_dict, hr_ecg_10s, threshold_list):
    MAE_results = {}
    for threshold in threshold_list:
        try:
            r_peaks_radar, lstm_probability = get_rpeaks(probability_dict, fs_radar=fs_radar, outlier_correction = True, threshold=threshold)
        except EcgProcessingError:
            MAE_results[threshold] = np.nan
            continue
            
        hr_radar = pd.DataFrame({"Heart_Rate": 60 / r_peaks_radar["RR_Interval"]})
        hr_radar.index = hr_radar.index.floor("10s")
        hr_radar_10s = hr_radar.groupby("date (Europe/Berlin)").mean()
        hr_radar_10s = hr_radar_10s.interpolate().rolling(20, center = True, min_periods=1).mean()
        
        hr_ecg_10s, hr_radar_10s = cut_signals(hr_ecg_10s, hr_radar_10s)
        
        MAE = abs(hr_ecg_10s - hr_radar_10s).mean()
        MAE_results[threshold] = MAE
    
    return MAE_results, lstm_probability

In [15]:
def get_hr_ecg(subj):
    ecg_data = subj.ecg_data.data_as_df(index="local_datetime")[["ECG II"]]
    ecg_data = ecg_data.rename(columns={"ECG II":"ecg"})
    ep = EcgProcessor(ecg_data, 256)
    ep.ecg_process(outlier_correction=None, clean_method = clean_method, peak_mathod=peak_method)
    hr_ecg = ep.heart_rate["Data"]
    hr_ecg.index = hr_ecg.index.floor("10s")
    hr_ecg_10s = hr_ecg.groupby("date (Europe/Berlin)").mean()
    hr_ecg_10s = hr_ecg_10s.rolling(20, center=True, min_periods=1).mean()

    return hr_ecg_10s


In [16]:
threshold_list = [0.050, 0.075, 0.1, 0.125, 0.150, 0.175, 0.200, 0.225, 0.250, 0.255, 0.260, 0.265, 0.270, 0.275, 0.280, 0.285, 0.290, 0.295, 0.300, 0.325, 0.350, 0.375, 0.400]
id_list = ["01", "02", "03", "04"]#, "05", "06", "07", "10", "11", "12", "14", "16", "18", "19","20","21","22","24", "25", "26", "27", "28", "29", "31", "32", "36", "37", "38", "41", "42", "43", "44"]

In [ ]:
subj_wise_MAE = {}
for subj in dataset:

    if str(subj.index["subj_id"][0]) not in id_list:
        continue

    file_path = "MAE_gridsearch_subj_" + str(subj.index["subj_id"][0]) + ".csv"
    
    # Check if the file exists
    file_exists = os.path.isfile(file_path)
    if file_exists:
        print("File for subj " + str(subj.index["subj_id"][0]) +  " already existis ... skip!")
        continue

    radar_data = subj.radar_data.data_as_df(index="local_datetime", add_sync_out=True)
    synced_radar = subj.sync_radar(radar_data)

    probability_dict = process_radar(synced_radar)

    hr_ecg_10s = get_hr_ecg(subj)
    
    MAE_dict, lstm_probabiliy = get_MAE_results(probability_dict, hr_ecg_10s, threshold_list)

    print("MAE of subj " + str(subj.index["subj_id"][0]))
    print(MAE_dict)

    pd.DataFrame(MAE_dict).to_csv("MAE_gridsearch_subj_120_" + subj.index["subj_id"][0] + ".csv")
    
    subj_wise_MAE[subj] = MAE_dict
    

Prepare SyncedDataset
Sync beginning of m-sequence


/Users/danielkrauss/code/Empkins/empkins-io/empkins_io/sync/_sync.py:325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_secondary.loc[:, sync_channel_secondary] = self._binarize_signal(


-4


/Users/danielkrauss/code/Empkins/empkins-io/empkins_io/sync/_sync.py:325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_secondary.loc[:, sync_channel_secondary] = self._binarize_signal(


-6


/Users/danielkrauss/code/Empkins/empkins-io/empkins_io/sync/_sync.py:325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_secondary.loc[:, sync_channel_secondary] = self._binarize_signal(


-30
Find shift at the end of the m-sequence
Shift: rad2_aligned_ 449
Shift: rad3_aligned_ 179
Shift: rad4_aligned_ 31
Resample sample-wise to get equal length


In [ ]:
subj_wise_MAE

In [ ]:
MAE_dict

In [ ]:
MAE_dict = get_MAE_results(probability_dict, hr_ecg_10s, threshold_list)

print("MAE of subj " + str(subj.index["subj_id"][0]))
print(MAE_dict)

In [ ]:
pd.DataFrame(MAE_dict).T